In [1]:
%matplotlib inline
import seaborn as sns, pandas as pd, numpy as np, os, qiime2 as q2
from matplotlib.pylab import plt

from biom import Table
from skbio.stats.ordination import cca, scale
from emperor import Emperor

def load_mf(fn, index='#SampleID'):
    _df = pd.read_csv(fn, sep='\t', dtype='str', na_values=[], keep_default_na=False)
    _df.set_index(index, inplace=True)
    return _df

In [2]:
cd trimmed-150nts/

/Users/yoshikivazquezbaeza/Documents/PDF/KnightLaboratory/HastyWater/trimmed-150nts


In [3]:
mf = load_mf('mapping-file.alpha.tsv')

In [4]:
NUMERIC = ['alkalinity', 'ammonian', 'cod', 'day',
           'nitraten', 'nitrogent', 'ph', 'phosphorousr',
           'phosphoroust', 'tss_vss', 'volatile_fatty_acids',
           'observed_otus', 'shannon', 'faith_pd']

NUMERIC_NO_ALPHA = ['alkalinity', 'ammonian', 'cod', 'day',
                    'nitraten', 'nitrogent', 'ph', 'phosphorousr',
                    'phosphoroust', 'tss_vss', 'volatile_fatty_acids']

Convert some of these columns into numeric types. Note that we don't include any alpha diversity metrics as CCA performs best with non-rdundant variables and non-correlated variables; most of the cases these variables are.

In [5]:
table = mf[NUMERIC_NO_ALPHA].copy()

for c in table.columns:
    table[c] = pd.to_numeric(table[c], errors='coerce')

# ordination methods won't work with missing data
table.dropna(inplace=True)

table.iloc[:, :] = scale(table.values, ddof=1)

In [6]:
bt = q2.Artifact.load('feature-table.even.18813.normalized.qza').view(Table).to_dataframe()
bt = bt.to_dense()
bt = bt[table.index].copy()
bt = bt.T

The environmental table cannot have negative values, so we add a constant so we can still use the scaled data. Type 1 scaling represents similarity amongst variables.

In [7]:
res = cca(y=table + 10, x=bt, scaling=1)

Emperor(res, mf)

/Users/yoshikivazquezbaeza/git_sw/emperor/emperor/util.py:737: EmperorWarning: Plots generated with `remote=True` using a development version of Emperor, may fail to load in the future (only the logo may be displayed instead of the plot). To avoid this, use a release version of Emperor.
  EmperorWarning)


If you want to save the plot outside of the notebook:

In [ ]:
viz = Emperor(res, mf, remote='./')
os.makedirs('cca', exist_ok=True)

with open(os.path.join('cca', 'index.html'), 'w') as f:
    f.write(viz.make_emperor(standalone=True))
    viz.copy_support_files('cca')